In [1]:
# данный ноутбут отслеживает резкие изменения цены или объема

/home/me/soft/other/python-bittrex-master/bittrex


In [ ]:
cd /home/me/soft/other/python-bittrex-master/bittrex/

In [2]:
from bittrex import Bittrex
import json
import codecs
import numpy as np
import time

In [4]:
bittrex = Bittrex(None, None)


'''
response = bittrex.get_markets()
resp_ancii = codecs.getreader('utf-8')(response)
for i in resp_ancii.values()[1]:
    print (i.get('MarketName'))
'''
market_names = []

sumary = bittrex.get_market_summaries() # первый запрос

for i in sumary.get('result'):
    market_names.append(i.get('MarketName'))
    
while 1:
    # инициализация переменных

    bids = np.array([])
    bids_new = np.array([])
    asks = np.array([])
    asks_new = np.array([])
    volums = np.array([])
    volums_new = np.array([])

    # заполнение памяти от первого запроса

    sumary = bittrex.get_market_summaries()
    for i in sumary.get('result'):
        bids = np.append(bids, i.get('Bid'))
        asks = np.append(asks, i.get('Ask'))
        volums = np.append(volums, i.get('Volume'))
        
    time.sleep(10) # задержка между запросами

    sumary_new = bittrex.get_market_summaries() # второй запрос

    ts = sumary_new.get('result')[60].get('TimeStamp') # метка времени

    # заполнение памяти от второго запроса

    for i in sumary_new.get('result'):
        bids_new = np.append(bids_new, i.get('Bid'))
        asks_new = np.append(asks_new, i.get('Ask'))
        volums_new = np.append(volums_new, i.get('Volume'))
    
    # вычисление разницы с предыдущим запросом

    delta_bids = bids_new/bids
    delta_bids = delta_bids[~np.isnan(delta_bids)]
    index_max_bid = np.where(delta_bids == delta_bids.max())
    
    
    delta_asks = asks_new/asks
    delta_asks = delta_asks[~np.isnan(delta_asks)]
    index_max_ask = np.where(delta_asks == delta_asks.max())
    
    
    delta_vols = volums_new/volums
    delta_vols = delta_vols[~np.isnan(delta_vols)]
    index_max_vol = np.where(delta_vols == delta_vols.max())
    
    
    # эвристика сигнала

    if delta_bids.max() >= 1.1 or delta_bids.max() <= 0.9:
        print ('BID ALERT!', market_names[index_max_bid[0][0]])
        #file.write('BID ALERT!')
        #file.write(market_names[index_max_bid[0][0]])

    if delta_asks.max() >= 1.1 or delta_asks.max() <= 0.9:
        print ('ASK ALERT!', market_names[index_max_ask[0][0]])
        #file.write('ASK ALERT!')
        #file.write(market_names[index_max_ask[0][0]])

    if delta_vols.max() >= 10:
        print ('VOL ALERT!', market_names[index_max_vol[0][0]])
        #file.write('VOL ALERT!')
        #file.write(market_names[index_max_vol[0][0]])

    print (ts)
    print ('max delta bid is', delta_bids.max(), market_names[index_max_bid[0][0]])
    print ('max delta ask is', delta_asks.max(), market_names[index_max_ask[0][0]])
    print ('max delta vol is', delta_vols.max(), market_names[index_max_vol[0][0]])

#    file.write(str(ts))
#    file.write(b)
#    file.write(a)
#    file.write(v)
#    file.write('################\n')
#                                                                          
#b = '\nmax delta bid is' + str(delta_bids.max()) + str(market_names[index_max_bid[0][0]]) + '\n'
#a = 'max delta ask is' + str(delta_asks.max()) + str(market_names[index_max_ask[0][0]]) + '\n'
#v = 'max delta vol is' + str(delta_vols.max()) + str(market_names[index_max_vol[0][0]]) + '\n'
#
#delta = {}
#for i,n in enumerate(market_names):
#    delta.setdefault(market_names[i], (base_volums_new[i]/base_volums[i], bids_new[i]/bids[i], asks_new[i]/asks[i]))
#print (delta)


2018-08-09T04:38:34.787
max delta bid is 1.00157418339 BTC-TIX
max delta ask is 1.00826446281 BTC-MFT
max delta vol is 1.00357396621 BTC-UP
2018-08-09T04:38:34.787
max delta bid is 1.00474462741 ETH-REP
max delta ask is 1.00694745983 BTC-MORE
max delta vol is 1.00031733036 BTC-BTG


KeyboardInterrupt: 